## 遺伝子は不連続である

**図1.** DNAの鋳型鎖上を移動しながらRNAP（RNAポリメラーゼ）がpre-mRNAを伸長させる様子。

**図2.** RNAはチミンがウラシルに置き換わる点を除き、DNAのコーディング鎖と同一である。

「DNAをRNAに転写する」において、DNAの一本鎖が転写の過程でRNAの一本鎖にコピーされることを述べましたが、転写がどのように行われるかについては触れていませんでした。

核内では、RNAポリメラーゼ（RNAP）と呼ばれる酵素（化学反応を加速させる分子）が転写を開始します。RNAPはまず、DNAの相補的塩基を結びつけている結合を切断します。その後、DNAの2本鎖のうち1本を「鋳型鎖」として利用しながらpre-mRNA（前駆体mRNA）と呼ばれる分子を合成します。RNAPが鋳型鎖上を進み次のヌクレオチドに出会うと、成長中のRNA鎖に相補的な塩基を付加します。ただし、チミンの代わりにウラシルを用いる必要があります（図1参照）。

RNAは相補性に基づいて構築されるため、もう一方のDNA鎖である「コーディング鎖」は、チミンがウラシルに置き換わる点を除き、新しく合成されたRNA鎖と同一です（図2参照。「DNAをRNAに転写する」を思い出してください）。

RNAPがRNAを数ヌクレオチド合成すると、最初に解離していたDNAの相補的塩基同士が再び結合します。全体として、この過程はDNA二重らせんをジッパーで開き、すぐに閉じていくようなイメージに似ています。その間にpre-mRNA鎖が合成されます。

さらに重要なのは、DNAのある部分配列全体が一気にRNAに転写され、その後コドンごとにペプチドへ翻訳されるわけではないという点です。実際には、pre-mRNAはまずイントロンとエキソンと呼ばれる小さな断片に切り分けられます。タンパク質翻訳のためにはイントロンは除去され、エキソンは順番に繋ぎ合わせられて最終的なmRNA鎖が作られます。この切断と結合の過程は「スプライシング」と呼ばれ、スプライソソームと呼ばれるRNAとタンパク質の複合体によって行われます。RNAとタンパク質からなるスプライソソームが、RNAをスプライシングしてタンパク質を作るという事実は、いわゆる「分子レベルの鶏が先か卵が先か」問題の一例であり、まだ完全には解明されていません。

DNAにおいて、ある遺伝子に由来するエキソン部分を総称して、その遺伝子の「コーディング領域」と呼びます。

---

### 問題

RNA配列中のエキソンとイントロンを特定したら、イントロンを削除し、残ったエキソンを連結して翻訳可能な新しい配列を作ればよい。

**与えられるもの:**  
DNA配列`s`（長さ最大1 kbp）と、その`s`中のイントロンに相当する部分配列の集合。すべての配列はFASTA形式で与えられる。

**求めるもの:**  
`s`のエキソン部分を転写・翻訳して得られるタンパク質配列を出力せよ。（与えられるデータセットに対しては解が一意に定まる。）

---

In [1]:
def read_fasta(fasta):
    sequences = {}
    header = None
    seq = []
    fasta = fasta.splitlines()
    for line in fasta:
        if line.startswith('>'):
            if header is not None:
                sequences[header] = ''.join(seq)
            header = line[1:]  # Remove '>'
            seq = []
        else:
            seq.append(line)
    if header is not None:
        sequences[header] = ''.join(seq)  # Add the last sequence
    return sequences

In [2]:
codon_dict = {
    "UUU": "F", "UUC": "F", "UUA": "L", "UUG": "L",
    "CUU": "L", "CUC": "L", "CUA": "L", "CUG": "L",
    "AUU": "I", "AUC": "I", "AUA": "I", "AUG": "M",
    "GUU": "V", "GUC": "V", "GUA": "V", "GUG": "V",
    "UCU": "S", "UCC": "S", "UCA": "S", "UCG": "S",
    "CCU": "P", "CCC": "P", "CCA": "P", "CCG": "P",
    "ACU": "T", "ACC": "T", "ACA": "T", "ACG": "T",
    "GCU": "A", "GCC": "A", "GCA": "A", "GCG": "A",
    "UAU": "Y", "UAC": "Y", "UAA": "Stop", "UAG": "Stop",
    "CAU": "H", "CAC": "H", "CAA": "Q", "CAG": "Q",
    "AAU": "N", "AAC": "N", "AAA": "K", "AAG": "K",
    "GAU": "D", "GAC": "D", "GAA": "E", "GAG": "E",
    "UGU": "C", "UGC": "C", "UGA": "Stop", "UGG": "W",
    "CGU": "R", "CGC": "R", "CGA": "R", "CGG": "R",
    "AGU": "S", "AGC": "S", "AGA": "R", "AGG": "R",
    "GGU": "G", "GGC": "G", "GGA": "G", "GGG": "G"
}

In [3]:
import re
def extract_cds(seqs:list[dict])->str:
    cds = ""
    for _, seq in seqs.items():
        if cds == "":
            cds = seq
        else:
            intron_len = len(seq)
            intron_start = re.search(seq,cds).start()
            cds = cds[:intron_start]+cds[intron_start+intron_len:]
    return cds            

In [4]:
def transcribe_rna_to_aa(seq):
    aa = []
    # escape stop codon
    for i in range(0, len(seq)-3, 3):
        codon = seq[i:i+3]
        codon = codon.replace("T", "U")
        aa.append(codon_dict.get(codon))
    aa = "".join(aa)
    return aa

In [5]:
sample_seqs = """>Rosalind_10
ATGGTCTACATAGCTGACAAACAGCACGTAGCAATCGGTCGAATCTCGAGAGGCATATGGTCACATGATCGGTCGAGCGTGTTTCAAAGTTTGCGCCTAG
>Rosalind_12
ATCGGTCGAA
>Rosalind_15
ATCGGTCGAGCGTGT"""

In [6]:
read_fasta(sample_seqs)

{'Rosalind_10': 'ATGGTCTACATAGCTGACAAACAGCACGTAGCAATCGGTCGAATCTCGAGAGGCATATGGTCACATGATCGGTCGAGCGTGTTTCAAAGTTTGCGCCTAG',
 'Rosalind_12': 'ATCGGTCGAA',
 'Rosalind_15': 'ATCGGTCGAGCGTGT'}

In [7]:
extract_cds(read_fasta(sample_seqs))

'ATGGTCTACATAGCTGACAAACAGCACGTAGCATCTCGAGAGGCATATGGTCACATGTTCAAAGTTTGCGCCTAG'

In [8]:
transcribe_rna_to_aa(extract_cds(read_fasta(sample_seqs)))

'MVYIADKQHVASREAYGHMFKVCA'